# RecordLinkage Linking

> Leveraging the RL library to determine approximate matching over a range of fields using various methods.

In [1]:
import json
import pandas
import time
import os
import recordlinkage
import csv
import core_constants as cc
import functions as fx

#not currently using jellyfish
import jellyfish as jf

## Set up the Dataframes

In [2]:
outputDir = '..//scrapedData//'
field_agg = "_"

## Load the source file dict
sourceFiles = json.loads(open('..//config//sourceFiles.json', "r").read())

## Load the id config
idConfig = json.loads(open('..//config//idConfigLink.json', "r").read())

In [3]:
## Cycle through all of files per source and drop into a list of dicts
## Printing list names for reference sake
dfs = []
for key in sourceFiles.keys():
    vars()[key] = fx.mergeSourceFiles (key, outputDir, sourceFiles)    
    dfs.append(vars()[key])

In [4]:
fx.createNewID(idConfig['sports247'], sports247, field_agg)
fx.createNewID(idConfig['rivals'], rivals, field_agg)

In [5]:
df_sports247 = pandas.DataFrame(sports247)
df_rivals = pandas.DataFrame(rivals)

In [6]:
df_sports247 = pandas.DataFrame(sports247).set_index('ID')
df_rivals = pandas.DataFrame(rivals).set_index('ID')
df_sports247.index.name = '247_ID'
df_sports247['ID'] = df_sports247.index
df_rivals.index.name = 'rivals_ID'
df_rivals['ID'] = df_rivals.index

## Import the Pickle file

In [17]:
features = pandas.read_pickle("features.pkl")
features.head()

ID  playerName  \
247_ID                     rivals_ID                                      
paulthurston_nebraska_2012 tommyarmstrong_nebraska_2012   0    0.285714   
                           alonzomoore_nebraska_2012      0    0.166667   
                           leroyalexander_nebraska_2012   0    0.000000   
                           jaredafalava_nebraska_2012     0    0.083333   
                           mohammedseisay_nebraska_2012   0    0.071429   

                                                         college      city  \
247_ID                     rivals_ID                                         
paulthurston_nebraska_2012 tommyarmstrong_nebraska_2012      1.0  0.000000   
                           alonzomoore_nebraska_2012         1.0  0.000000   
                           leroyalexander_nebraska_2012      1.0  0.166667   
                           jaredafalava_nebraska_2012        1.0  0.166667   
                           mohammedseisay_nebraska_2012      1.0  0.000000   

                                                         state  highSchool  \
247_ID                     rivals_ID                                         
paulthurston_nebraska_2012 tommyarmstrong_nebraska_2012      0    0.090909   
                           alonzomoore_nebraska_2012         0    0.125000   
                           leroyalexander_nebraska_2012      0    0.090909   
                           jaredafalava_nebraska_2012        0    0.090909   
                           mohammedseisay_nebraska_2012      0    0.066667   

                                                         position  year  \
247_ID                     rivals_ID                                      
paulthurston_nebraska_2012 tommyarmstrong_nebraska_2012         0     1   
                           alonzomoore_nebraska_2012            0     1   
                           leroyalexander_nebraska_2012         0     1   
                           jaredafalava_nebraska_2012           0     1   
                           mohammedseisay_nebraska_2012         0     1   

                                                              sum  
247_ID                     rivals_ID                               
paulthurston_nebraska_2012 tommyarmstrong_nebraska_2012  0.339518  
                           alonzomoore_nebraska_2012     0.327381  
                           leroyalexander_nebraska_2012  0.322511  
                           jaredafalava_nebraska_2012    0.334416  
                           mohammedseisay_nebraska_2012  0.305442

In [9]:
c = 0

for row in features.itertuples():
    if (c < 1):
        print (row)
        c = 1
    else:
        break
        
    


Pandas(Index=('paulthurston_nebraska_2012', 'tommyarmstrong_nebraska_2012'), ID=0, playerName=0.2857142857142857, college=1.0, city=0.0, state=0, highSchool=0.09090909090909094, position=0, year=1, sum=0.3395176252319109)


In [18]:
features.reset_index(inplace=True)
features.head()

,247_ID,rivals_ID,ID,playerName,college,city,state,highSchool,position,year,sum
0,paulthurston_nebraska_2012,tommyarmstrong_nebraska_2012,0,0.285714,1.0,0.000000,0,0.090909,0,1,0.339518
1,paulthurston_nebraska_2012,alonzomoore_nebraska_2012,0,0.166667,1.0,0.000000,0,0.125000,0,1,0.327381
2,paulthurston_nebraska_2012,leroyalexander_nebraska_2012,0,0.000000,1.0,0.166667,0,0.090909,0,1,0.322511
3,paulthurston_nebraska_2012,jaredafalava_nebraska_2012,0,0.083333,1.0,0.166667,0,0.090909,0,1,0.334416
4,paulthurston_nebraska_2012,mohammedseisay_nebraska_2012,0,0.071429,1.0,0.000000,0,0.066667,0,1,0.305442


In [21]:
unique247IDs = features['247_ID'].unique()

In [22]:
for i in unique247IDs:
    print(i)

paulthurston_nebraska_2012
gregmcmullen_nebraska_2012
jordanwesterkamp_nebraska_2012
imanicross_nebraska_2012
tommyarmstrong_nebraska_2012
michaelroseivey_nebraska_2012
jaredafalava_nebraska_2012
mohammedseisay_nebraska_2012
alonzomoore_nebraska_2012
vincentvalentine_nebraska_2012
thomasbrown_nebraska_2012
aaroncurry_nebraska_2012
zaireanderson_nebraska_2012
averymoss_nebraska_2012
samcotton_nebraska_2012
coreywhitaker_nebraska_2012
leroyalexander_nebraska_2012
lamarrwoodley_michigan_2003
prescottburgess_michigan_2003
shawncrable_michigan_2003
jimpresley_michigan_2003
ryanmundy_michigan_2003
willpaul_michigan_2003
claytonrichard_michigan_2003
jakelong_michigan_2003
jeromejackson_michigan_2003
adamkraus_michigan_2003
leonhall_michigan_2003
jeffzuttah_michigan_2003
garrettrivas_michigan_2003
antoncampbell_michigan_2003
patsharrow_michigan_2003
brandentenglemon_michigan_2003
dariushamilton_rutgers_2012
leontecarroo_rutgers_2012
chrismuller_rutgers_2012
jjdenman_rutgers_2012
quanzelllamber

brandonrussell_rutgers_2014
georgebehr_rutgers_2014
isaiahwharton_rutgers_2014
dariandailey_rutgers_2014
samblue_rutgers_2014
alanlucy_rutgers_2014
ericmargolis_rutgers_2014
sandyanya_rutgers_2014
charlesscarff_rutgers_2014
michaelcintron_rutgers_2014
kevindaly_rutgers_2014
talibabdurraoof_rutgers_2014
justinnelson_rutgers_2014
sidneygopre_rutgers_2014
davidbarrent_michiganstate_2009
edwinbaker_michiganstate_2009
chrisnorman_michiganstate_2009
andrewmaxwell_michiganstate_2009
larrycaper_michiganstate_2009
dionsims_michiganstate_2009
donaldspencer_michiganstate_2009
blaketreadwell_michiganstate_2009
nateklatt_michiganstate_2009
jeremygainer_michiganstate_2009
tyquanhammock_michiganstate_2009
jairusjones_michiganstate_2009
patrickwhite_michiganstate_2009
benniefowler_michiganstate_2009
denicosallen_michiganstate_2009
danfrance_michiganstate_2009
henryconway_michiganstate_2009
micajahreynolds_michiganstate_2009
danadixon_michiganstate_2009
denzeldrone_michiganstate_2009
jerelworthy_michig

rickyhenry_nebraska_2006
sethjensen_nebraska_2006
mikemcneill_nebraska_2006
pierreallen_nebraska_2006
anthonywest_nebraska_2006
willhenry_nebraska_2006
coreyyoung_nebraska_2006
brandonjohnson_nebraska_2006
victoryhaines_nebraska_2006
keithwilliams_nebraska_2006
cruzbarrette_nebraska_2006
mikesmith_nebraska_2006
benmartin_nebraska_2006
majorculbert_nebraska_2006
rickeythenarse_nebraska_2006
kordellyoung_rutgers_2006
timbrown_rutgers_2006
jeffminemyer_rutgers_2006
ryandimperio_rutgers_2006
georgejohnson_rutgers_2006
antoniolowery_rutgers_2006
shamargraves_rutgers_2006
matthardison_rutgers_2006
kevinholloway_rutgers_2006
zairekitchen_rutgers_2006
chrispauletienne_rutgers_2006
jessecisco_rutgers_2006
blairbines_rutgers_2006
jackcorcoran_rutgers_2006
soriebayoh_rutgers_2006
edmondlaryea_rutgers_2006
charlienoonan_rutgers_2006
markuswhite_rutgers_2006
molange_rutgers_2006
ericwilson_rutgers_2006
andresmorales_rutgers_2006
marlonromulus_rutgers_2006
patnemorin_rutgers_2006
tomlang_rutgers_200

ahongalufusimalohi_kansas_2013
benjohnson_kansas_2013
ishmaelhyman_kansas_2013
markthomas_kansas_2013
mikesmithburg_kansas_2013
tedarianjohnson_kansas_2013
colinspencer_kansas_2013
zachfondal_kansas_2013
samsonfaifili_kansas_2013
isaiahjohnson_kansas_2013
montellcozart_kansas_2013
trevorpardula_kansas_2013
michaelmesh_kansas_2013
petergallo_kansas_2013
brandonholloman_kansas_2013
michaelwhitfield_kansas_2013
tjsemke_kansas_2013
johnwirtel_kansas_2013
cameronrosser_kansas_2013
jacobdavis_kansas_2013
jordangoldenberg_kansas_2013
kylepullia_kansas_2013
chrismartin_kansas_2013
zackaryyoung_kansas_2013
chaunceyclark_texastech_2003
danchristian_texastech_2003
phillipdaugherty_texastech_2003
antoniohuffman_texastech_2003
keyuntadawson_texastech_2003
sionehavili_texastech_2003
darylancarroll_texastech_2003
michaelbrisco_texastech_2003
bengriffin_texastech_2003
joeygarcia_texastech_2003
mattgrissel_texastech_2003
dariusjones_baylor_2009
ivorywade_baylor_2009
glascomartin_baylor_2009
skylerscott

barrettmatthews_texas_2009
padenkelley_texas_2009
calvinhowell_texas_2009
garrettporter_texas_2009
kennyvaccaro_texas_2009
eryonbarnett_texas_2009
derekjohnson_texas_2009
tevinmims_texas_2009
treygraham_texas_2009
kylekriegel_texas_2009
dominiquejones_texas_2009
patricknkwopara_texas_2009
tyreekhill_oklahomastate_2014
devonthomas_oklahomastate_2014
masonrudolph_oklahomastate_2014
jameswashington_oklahomastate_2014
devanteaverette_oklahomastate_2014
juwannoffray_oklahomastate_2014
jordanfrazier_oklahomastate_2014
luketimian_oklahomastate_2014
taylorcornelius_oklahomastate_2014
gyasiakem_oklahomastate_2014
keenenbrown_oklahomastate_2014
jordanbrailford_oklahomastate_2014
chrishardeman_oklahomastate_2014
treycarter_oklahomastate_2014
joshmabin_oklahomastate_2014
jarrellowens_oklahomastate_2014
sionepalelei_oklahomastate_2014
kirktucker_oklahomastate_2014
joyanwilliams_oklahomastate_2014
chrislacy_oklahomastate_2014
justinphillips_oklahomastate_2014
ramonrichards_oklahomastate_2014
matthew

...

mackayphillips_army_2017
nickstokes_army_2017
emmanuelukhueligbe_army_2017
keshaunwells_army_2017
christiananderson_army_2017
mylesbell_uconn_2019
ericwatts_uconn_2019
justinmoore_uconn_2019
mattdrayton_uconn_2019
cameronross_uconn_2019
dillonharris_uconn_2019
jaycemedlock_uconn_2019
malikdixon_uconn_2019
jeffwoods_uconn_2019
chaselundt_uconn_2019
jacksonmitchell_uconn_2019
jackzergiotis_uconn_2019
rayontebrown_uconn_2019
kevensclercius_uconn_2019
winstonjules_uconn_2019
dalmontgourdine_uconn_2019
djhord_notredame_2005
joeyhiben_notredame_2005
paulduncan_notredame_2005
michaelturkovich_notredame_2005
kevinwashington_notredame_2005
scottsmith_notredame_2005
evansharpley_notredame_2005
davidbruton_notredame_2005
rayherring_notredame_2005
patrickkuntz_notredame_2005
davidgrimes_notredame_2005
kylemccarthy_notredame_2005
derrellhand_notredame_2005
asaphschwapp_notredame_2005
stevequinn_notredame_2005
kylevannoy_byu_2009
craigbills_byu_2009
richardwilson_byu_2009
terryalletto_byu_2009
adamh

bilalelamin_marshall_2004
douglegursky_marshall_2004
samwalker_marshall_2004
demonegoins_marshall_2004
ianhoskins_marshall_2004
jasonkaminski_marshall_2004
jonmoravec_marshall_2004
mandellsmith_marshall_2004
blakemerritt_marshall_2004
floydwright_marshall_2004
emmanuelspann_marshall_2004
tonyleath_marshall_2004
matthewparkhurst_marshall_2004
drewmelancon_louisianatech_2016
williebaker_louisianatech_2016
kaderrionmason_louisianatech_2016
jackabraham_louisianatech_2016
devinhamilton_louisianatech_2016
zachcousar_louisianatech_2016
davidbeasley_louisianatech_2016
taylorfondal_louisianatech_2016
kalanritchie_louisianatech_2016
ljariussneed_louisianatech_2016
israeltucker_louisianatech_2016
erickendzior_louisianatech_2016
reggiecleveland_louisianatech_2016
damarionking_louisianatech_2016
kennedymadison_louisianatech_2016
zachhannibal_louisianatech_2016
kodyrussey_louisianatech_2016
keonatyegarner_louisianatech_2016
joshuaoutlaw_louisianatech_2016
camronharrell_southernmississippi_2020
frank

dedricksimpson_utep_2016
kevindove_utep_2016
jaredgoldwire_utep_2016
dylanparsee_utep_2016
ericfoster_utep_2016
ronaldawatt_utep_2016
sionetupou_utep_2016
eddiejohnson_louisianatech_2013
nickthomason_louisianatech_2013
hakimgray_louisianatech_2013
terrellpinson_louisianatech_2013
xavierwoods_louisianatech_2013
carloshenderson_louisianatech_2013
mitchellbell_louisianatech_2013
trenttaylor_louisianatech_2013
cammanning_louisianatech_2013
deangelobrooks_louisianatech_2013
thomasmcdonald_louisianatech_2013
nicholasbickham_louisianatech_2013
mitchellvillemez_louisianatech_2013
michaeljacob_louisianatech_2013
chrisaye_louisianatech_2013
loganmcpherson_louisianatech_2013
jonathanbarnes_louisianatech_2013
trecarter_louisianatech_2013
kevingary_louisianatech_2013
bostonscott_louisianatech_2013
tuckeretheridge_louisianatech_2013
dariushodge_marshall_2017
kobycumberlander_marshall_2017
artisjohnson_marshall_2017
kenardking_marshall_2017
nequanrenalds_marshall_2017
chadclay_marshall_2017
monterioh

willchandler_unlv_2007
lukeplante_unlv_2007
brantmartinez_unlv_2007
adleyeshraghipour_unlv_2007
brycesaldi_unlv_2007
yusefrodgers_unlv_2007
robertlinda_unlv_2007
mattmurphy_unlv_2007
ericpinkins_sandiegostate_2009
natberhe_sandiegostate_2009
ronniehillman_sandiegostate_2009
marvinjones_sandiegostate_2009
anthonymiller_sandiegostate_2009
marcusandrews_sandiegostate_2009
coltonmorrison_sandiegostate_2009
jacobdriver_sandiegostate_2009
osmondnicholas_sandiegostate_2009
aareinbooker_sandiegostate_2009
gabelemon_sandiegostate_2009
brandonchambers_sandiegostate_2009
jakebernards_sandiegostate_2009
gavinescobar_sandiegostate_2009
khalidstevens_sandiegostate_2009
leonmcfadden_sandiegostate_2009
nicktenhaeff_sandiegostate_2009
colinlockett_sandiegostate_2009
walterkazee_sandiegostate_2009
prestonking_sandiegostate_2009
colinshumate_sandiegostate_2008
jimmymiller_sandiegostate_2008
alecjohnson_sandiegostate_2008
drewwestling_sandiegostate_2008
erikquinones_sandiegostate_2008
djshields_sandiegost

vershadjackson_newmexico_2012
marquisbundy_newmexico_2012
ryansantos_newmexico_2012
joshbaggett_newmexico_2012
christianrebhun_newmexico_2012
davidorvick_newmexico_2012
cranstonjones_newmexico_2012
mikearrendondo_newmexico_2012
rashadrainey_newmexico_2012
paytronhightower_newmexico_2012
sagwanedwards_newmexico_2012
jamarcusyoung_newmexico_2012
chrisedling_newmexico_2012
darianallen_newmexico_2012
johnnyvizcaino_newmexico_2012
dustinmoore_newmexico_2004
rodneyferguson_newmexico_2004
kennethshelton_newmexico_2004
adamgarday_newmexico_2004
donaldpaul_newmexico_2004
lukehiggins_newmexico_2004
kenwest_newmexico_2004
siosifavave_newmexico_2004
bogreer_newmexico_2004
blakeligon_newmexico_2004
steveharris_newmexico_2004
anthonycarter_newmexico_2004
deandrewright_newmexico_2004
yiminbauman_newmexico_2004
freddiemcgowen_newmexico_2004
chrismark_newmexico_2004
kevinbalogun_newmexico_2004
maverickethridge_newmexico_2004
chrisbrawley_newmexico_2004
jakebowe_newmexico_2004
matthewquillen_newmexico_2

In [ ]:
## This is absolutely necessary if you want to access the IDs through a dictionary.  
## Without it the 247_ID and rivals_ID won't carry forward
#features.reset_index(inplace=True)
#features.head()
#features = features.to_dict('records')

In [8]:
df_sports247.reset_index(inplace=True)

In [9]:
sportDict = df_sports247.to_dict('records')

In [23]:
finalSet = []
for record in sportDict:
    finalRecord = {}
    for key,value in record.items():
        finalRecord[key] = value
    try:
        multiResults = features.loc[record['247_ID']]
        rivalsID = multiResults['sum'].idxmax()
        
        if (multiResults['ID'].max() == 1):
            finalRecord['confidence'] = 1
        else:
            finalRecord['confidence'] = multiResults['sum'].max()
    except:
        finalRecord['error'] = 'No feature found with that ID'

    try:
        rivalsResult = (df_rivals.loc[rivalsID]).to_dict()
        for key, value in rivalsResult.items():
            finalRecord['r_' + key] = value
    except:
        finalRecord['error'] = 'No rivals match found'
    finalSet.append(finalRecord)


In [24]:
df_finalSet = pandas.DataFrame(finalSet)

In [25]:
df_finalSet

,247_ID,school,year,playerName,highSchool,city,state,position,height,weight,...,r_position,r_height,r_weight,r_stars,r_nationalRank,r_positionRank,r_stateRank,r_displayName,r_ID,error
0,paulthurston_nebraska_2012,nebraska,2012,paulthurston,Arvada West,Arvada,CO,OT,77,275,...,OL,77.0,274.0,4,147.0,14.0,3.0,Paul Thurston,paulthurston_nebraska_2012,NaN
1,gregmcmullen_nebraska_2012,nebraska,2012,gregmcmullen,Archbishop Hoban,Akron,OH,SDE,77,255,...,DE,77.0,255.0,4,192.0,12.0,11.0,Greg McMullen,gregmcmullen_nebraska_2012,NaN
2,jordanwesterkamp_nebraska_2012,nebraska,2012,jordanwesterkamp,Montini Catholic,Lombard,IL,WR,73,190,...,WR,74.0,195.0,4,NaN,43.0,8.0,Jordan Westerkamp,jordanwesterkamp_nebraska_2012,NaN
3,imanicross_nebraska_2012,nebraska,2012,imanicross,North Hall,Gainesville,GA,RB,73,222,...,RB,73.0,220.0,3,NaN,25.0,34.0,Imani Cross,imanicross_nebraska_2012,NaN
4,tommyarmstrong_nebraska_2012,nebraska,2012,tommyarmstrong,Steele,Cibolo,TX,DUAL,73,200,...,QB,74.0,210.0,3,NaN,9.0,49.0,Tommy Armstrong,tommyarmstrong_nebraska_2012,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50918,brettmedforth_troy_2013,troy,2013,brettmedforth,Sylacauga Sch,Sylacauga,AL,OG,75,245,...,DT,73.0,275.0,3,NaN,NaN,NaN,Arie Anderson,arieanderson_troy_2013,NaN
50919,cardelllue_troy_2013,troy,2013,cardelllue,Hebron,Carrollton,TX,CB,72,185,...,DB,72.0,185.0,2,NaN,NaN,NaN,Cardell Lue,cardelllue_troy_2013,NaN
50920,zachmoore_troy_2013,troy,2013,zachmoore,Coffee,Douglas,GA,ILB,73,200,...,LB,73.0,200.0,2,NaN,NaN,NaN,Zach Moore,zachmoore_troy_2013,NaN
50921,sethroberts_troy_2013,troy,2013,sethroberts,Ariton Sch,Ariton,AL,WDE,76,250,...,DT,76.0,250.0,2,NaN,NaN,NaN,Seth Roberts,sethroberts_troy_2013,NaN


In [27]:
df_finalSet.to_csv('review.csv')  